In [ ]:
'''
import os
os.environ["SPARK_HOME"] = "/home/osboxes/spark/spark-2.4.0-bin-hadoop2.7/"

import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.functions import levenshtein 
#import pyspark.sql.HiveContext as HiveContext
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#conf = pyspark.SparkConf().setAppName('appName').setMaster('spark://192.168.11.128:8080')

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

#spark.enableHiveSupport()
#import spark.sql.hive.HiveContext as HiveContext
#HiveContext sqlContext = spark.sql.hive.HiveContext(sc.sc())

'''


In [1]:
from os.path import expanduser, join, abspath
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

Py4JError: org.apache.spark.api.python.PythonUtils.getEncryptionEnabled does not exist in the JVM

In [ ]:
df_weeklyTop200 = spark.read.csv('../datasets/spotify_top_200_weekly.csv', header=True)
df_trackDetails = spark.read.csv('../datasets/spotify_track_details.csv', header=True)

In [ ]:
spark.read.csv('../datasets/spotify_top_200_weekly.csv', header=True).withColumnRenamed("Track Name","Track_Name").write.mode('Overwrite').saveAsTable('spotify_top_200_weekly')

In [ ]:
df_weeklyTop200 = df_weeklyTop200.withColumnRenamed("Track Name","Track_Name")


In [ ]:
df_weeklyTop200.write.mode('Overwrite').saveAsTable('spotify_top_200_weekly')
df_trackDetails.write.mode('Overwrite').saveAsTable('spotify_track_details')

In [ ]:
spark.sql('select * from spotify_track_details limit 2').show()

In [ ]:
df_trackDetails.columns

In [ ]:
rs = spark.sql('select * from spotify_track_details')
rs.count()

In [ ]:
spark.sql("select * from spotify_top_200_weekly stw \
where Position > 0  and Position <= 10 and region = 'gb'")\
.createOrReplaceTempView('spotify_top_10_weekly')

In [ ]:
spark.sql("select count('position'), Artist from spotify_top_10_weekly stw \
where stw.region = 'gb' group by Artist")\
.createOrReplaceTempView('vw_unique_uk_artist')

In [ ]:
spark.sql("select count(artist_mbid), artist_spotifyid , artist_mbid,  artist_name \
from spotify_track_details std where artist_mbid <> '' \
group by artist_spotifyid , artist_mbid,  artist_name") \
.createOrReplaceTempView('vw_unique_artist_mbid')


In [ ]:
#rs = spark.sql("select * from vw_unique_uk_artist a left join vw_unique_artist_mbid b on a." + """Artist""" + " = b.artist_name where b.artist_spotifyid is null order by 2")

rs = spark.sql("select * from vw_unique_uk_artist a \
left join vw_unique_artist_mbid b on a.Artist = b.artist_name \
where b.artist_spotifyid is not null order by 2")


In [ ]:
rs.count()

## Read saved tables from spark datawarehouse

In [ ]:
df = spark.read.parquet('spark-warehouse/weeklytop200/')

In [ ]:
df.createOrReplaceTempView('spotify_top_200_weekly')

In [ ]:
spark.sql("select * from spotify_top_200_weekly stw \
where Position > 0  and Position <= 10 and region = 'gb'").createOrReplaceTempView('spotify_top_10_weekly')

In [ ]:
spark.sql('select count(1) from spotify_top_10_weekly').show()

## Load data from csv file

In [ ]:
#create table and load data from csv
spark.sql("CREATE TABLE IF NOT EXISTS spotify_track_details ( \
    track_spotifyid STRING, \
    artist_spotifyid STRING,\
    artist_mbid STRING,\
    artist_name STRING,\
    track_url STRING,\
    track_popularity STRING,\
    track_duration_ms STRING,\
    track_is_local STRING,\
    albumid STRING,\
    album_track_number STRING,\
    album_release_date STRING,\
    album_type STRING) USING hive")

In [ ]:
spark.sql("LOAD DATA LOCAL INPATH '../datasets/spotify_track_details.csv' INTO TABLE spotify_track_details")

In [ ]:
spark.sql('select track_spotifyid,artist_spotifyid, artist_mbid, track_popularity from spotify_track_details').show(2, truncate=False)

In [ ]:
# Note: JDBC loading and saving can be achieved via either the load/save or jdbc methods
# Loading data from a JDBC source
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .load()

jdbcDF2 = spark.read \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

# Specifying dataframe column data types on read
jdbcDF3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .option("customSchema", "id DECIMAL(38, 0), name STRING") \
    .load()

# Saving data to a JDBC source
jdbcDF.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .save()

jdbcDF2.write \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

# Specifying create table column data types on write
jdbcDF.write \
    .option("createTableColumnTypes", "name CHAR(64), comments VARCHAR(1024)") \
    .jdbc("jdbc:postgresql:dbserver", "schema.tablename",
          properties={"user": "username", "password": "password"})

In [ ]:
   
dataframe = spark.read.format('jdbc').options(
    url = "jdbc:postgresql://localhost/public?user=postgres&password=postgres",
    database='public',
    dbtable='spotify_top_200_weekly'
).load()

dataframe.show()

In [ ]:
spark.sql('CREATE TABLE spotify_top_200_weekly (Position varchar(32767) NULL,spotify_id varchar(32767) NULL)')

In [ ]:
from pyspark.sql import SQLContext
sc = spark.sparkContext
sqlContext = SQLContext(sc, sparkSession=spark, jsqlContext=None)
url = 'jdbc:postgresql://localhost/gig_stg?user=postgres&password=postgres'
df = sqlContext.read.jdbc(url=url, table='spotify_top_weekly_200')

In [ ]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql:dbserver") \
    .option("dbtable", "schema.tablename") \
    .option("user", "username") \
    .option("password", "password") \
    .load()

In [ ]:
sc.stop()

In [ ]:
import os
os.environ["SPARK_HOME"] = "/home/osboxes/spark/spark-2.4.0-bin-hadoop2.7/"

import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import levenshtein 

#conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#conf = pyspark.SparkConf().setAppName('appName').setMaster('spark://192.168.11.128:8080')

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "/home/osboxes/postgresql-42.2.11.jar") \
    .getOrCreate()



#${url}	connection URL	jdbc:postgresql://osboxes:5432/postgres

In [ ]:
df = spark.read.format("jdbc")
    .option("url", "jdbc:postgresql://osboxes:5432/postgres")
    .option('driver', '')
    .option("dbtable", "spotify_top_200_weekly")
    .option("user", "postgres")
    .load()

df.printSchema()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.driver.extraClassPath', '/home/osboxes/postgresql-42.2.11.jar').getOrCreate()
url = 'jdbc:postgresql://108.167.159.189:5432/postgres'
properties = {'user': 'postgres', 'password': 'postgres'}
df = spark.read.jdbc(url=url, table='spotify_top_200_weekly', properties=properties)